# Preprocessing Text

In [2]:
import pandas as pd
from nltk.tokenize import ToktokTokenizer
import string
from functools import reduce
import pandas as pd
import unicodedata
from pandarallel import pandarallel
from sklearn.feature_extraction import text as txt

import os

## Defining functions

In [2]:
canada_stopwords = ['member','members','government','governments','opposition','opposite','leader',
    'hon','exminister','prime','minister','ministers','parliament','house',
    'ask','asked','asks','question','questioned','questions','bills','bill',
    'party','parties','mp','mps','sir','madam','mr','gentleman','gentlemen','lady','ladies',
    'speaker','chair','motion','motions','vote','votes','order','yes','deputy','secretary',
    'canada','canadian','canadians',
    'pursuant','supply','supplementary','please','friend','s',
    'clause','amendment','i','ii','iii','section','sections', 'colleague', 'colleagues'] + list(txt.ENGLISH_STOP_WORDS)

In [3]:
contractions = {"you'd": 'you would', "he'd": 'he would', "she's": 'she is', "where'd": 'where did', "might've": 'might have', "he'll": 'he will', "they'll": 'they will',  "mightn't": 'might not', "you'd've": 'you would have', "shan't": 'shall not', "it'll": 'it will', "mayn't": 'may not', "couldn't": 'could not', "they'd": 'they would', "so've": 'so have', "needn't've": 'need not have', "they'll've": 'they will have', "it's": 'it is', "haven't": 'have not', "didn't": 'did not', "y'all'd": 'you all would', "needn't": 'need not', "who'll": 'who will', "wouldn't've": 'would not have', "when's": 'when is', "will've": 'will have', "it'd've": 'it would have', "what'll": 'what will', "that'd've": 'that would have', "y'all're": 'you all are', "let's": 'let us', "where've": 'where have', "o'clock": 'oclock', "when've": 'when have', "what're": 'what are', "should've": 'should have', "you've": 'you have', "they're": 'they are', "aren't": 'are not', "they've": 'they have', "it'd": 'it would', "i'll've": 'i will have', "they'd've": 'they would have', "you'll've": 'you will have', "wouldn't": 'would not', "we'd": 'we would', "hadn't've": 'had not have', "weren't": 'were not', "i'd": 'i would', "must've": 'must have', "what's": 'what is', "mustn't've": 'must not have', "what'll've": 'what will have', "ain't": 'aint', "doesn't": 'does not', "we'll": 'we will', "i'd've": 'i would have', "we've": 'we have', "oughtn't": 'ought not', "you're": 'you are', "who'll've": 'who will have', "shouldn't": 'should not', "can't've": 'cannot have', "i've": 'i have', "couldn't've": 'could not have', "why've": 'why have', "what've": 'what have', "can't": 'cannot', "don't": 'do not', "that'd": 'that would', "who's": 'who is', "would've": 'would have', "there'd": 'there would', "shouldn't've": 'should not have', "y'all": 'you all', "mustn't": 'must not', "she'll": 'she will', "hadn't": 'had not', "won't've": 'will not have', "why's": 'why is', "'cause": 'because', "wasn't": 'was not', "shan't've": 'shall not have', "ma'am": 'madam', "hasn't": 'has not', "to've": 'to have', "how'll": 'how will', "oughtn't've": 'ought not have', "he'll've": 'he will have', "we'd've": 'we would have', "won't": 'will not', "could've": 'could have', "isn't": 'is not', "she'll've": 'she will have', "we'll've": 'we will have', "you'll": 'you will', "who've": 'who have', "there's": 'there is', "y'all've": 'you all have', "we're": 'we are', "i'll": 'i will', "i'm": 'i am', "how's": 'how is', "she'd've": 'she would have', "sha'n't": 'shall not', "there'd've": 'there would have', "he's": 'he is', "it'll've": 'it will have', "that's": 'that is', "y'all'd've": 'you all would have', "he'd've": 'he would have', "how'd": 'how did', "where's": 'where is', "so's": 'so as', "she'd": 'she would', "mightn't've": 'might not have'}


In [4]:
def clean_text(text):
    pandarallel.initialize(progress_bar=True,nb_workers=8, verbose=False)
    tk = ToktokTokenizer()
    
    # replace contractions
    text = reduce(lambda a, kv: a.replace(*kv), contractions.items(), text.lower())
    text = text.replace('\t',' ').replace('\n',' ').replace('\r',' ')
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    tokens = tk.tokenize(text)
    
    # replace stopwords and digits
    tokens = [w for w in tokens if w not in canada_stopwords and len(w)>2 and w!=' ' and not w.isdigit()]
    return ' '.join(tokens)

In [5]:
def preprocess_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"Processing {file_path}")
                preprocess_csv(file_path)

def preprocess_csv(csv_file):
    df = pd.read_csv(csv_file)
    df['speechtext_preprocessed'] = df['speechtext'].apply(lambda x: clean_text(x)if isinstance(x, str) else "")
    df.to_csv(csv_file, index=False)

## Clean Canadian Corpus

In [ ]:
df['cleaned_text'] = df['speech_text'].parallel_apply(clean_text)

In [29]:
root_folder = 'corpus_data/lipad'
preprocess_folder(root_folder)

Processing corpus_data/lipad/2013/11/2013-11-8.csv
Processing corpus_data/lipad/2013/11/2013-11-22.csv
Processing corpus_data/lipad/2013/11/2013-11-20.csv
Processing corpus_data/lipad/2013/11/2013-11-21.csv
Processing corpus_data/lipad/2013/11/2013-11-19.csv
Processing corpus_data/lipad/2013/11/2013-11-25.csv
Processing corpus_data/lipad/2013/11/2013-11-18.csv
Processing corpus_data/lipad/2013/11/2013-11-26.csv
Processing corpus_data/lipad/2013/11/2013-11-27.csv
Processing corpus_data/lipad/2013/11/2013-11-29.csv
Processing corpus_data/lipad/2013/11/2013-11-28.csv
Processing corpus_data/lipad/2013/11/2013-11-6.csv
Processing corpus_data/lipad/2013/11/2013-11-7.csv
Processing corpus_data/lipad/2013/11/2013-11-5.csv
Processing corpus_data/lipad/2013/11/2013-11-4.csv
Processing corpus_data/lipad/2013/6/2013-6-18.csv
Processing corpus_data/lipad/2013/6/2013-6-19.csv
Processing corpus_data/lipad/2013/6/2013-6-6.csv
Processing corpus_data/lipad/2013/6/2013-6-7.csv
Processing corpus_data/lipa

Processing corpus_data/lipad/2014/10/2014-10-27.csv
Processing corpus_data/lipad/2014/10/2014-10-31.csv
Processing corpus_data/lipad/2014/10/2014-10-30.csv
Processing corpus_data/lipad/2014/10/2014-10-24.csv
Processing corpus_data/lipad/2014/10/2014-10-20.csv
Processing corpus_data/lipad/2014/10/2014-10-21.csv
Processing corpus_data/lipad/2014/10/2014-10-23.csv
Processing corpus_data/lipad/2014/10/2014-10-3.csv
Processing corpus_data/lipad/2014/10/2014-10-2.csv
Processing corpus_data/lipad/2014/10/2014-10-1.csv
Processing corpus_data/lipad/2014/10/2014-10-6.csv
Processing corpus_data/lipad/2014/10/2014-10-7.csv
Processing corpus_data/lipad/2014/4/2014-4-7.csv
Processing corpus_data/lipad/2014/4/2014-4-4.csv
Processing corpus_data/lipad/2014/4/2014-4-1.csv
Processing corpus_data/lipad/2014/4/2014-4-3.csv
Processing corpus_data/lipad/2014/4/2014-4-2.csv
Processing corpus_data/lipad/2014/4/2014-4-30.csv
Processing corpus_data/lipad/2014/4/2014-4-28.csv
Processing corpus_data/lipad/2014/4/

Processing corpus_data/lipad/2012/9/2012-9-20.csv
Processing corpus_data/lipad/2012/9/2012-9-21.csv
Processing corpus_data/lipad/2012/11/2012-11-23.csv
Processing corpus_data/lipad/2012/11/2012-11-22.csv
Processing corpus_data/lipad/2012/11/2012-11-20.csv
Processing corpus_data/lipad/2012/11/2012-11-21.csv
Processing corpus_data/lipad/2012/11/2012-11-19.csv
Processing corpus_data/lipad/2012/11/2012-11-30.csv
Processing corpus_data/lipad/2012/11/2012-11-8.csv
Processing corpus_data/lipad/2012/11/2012-11-26.csv
Processing corpus_data/lipad/2012/11/2012-11-27.csv
Processing corpus_data/lipad/2012/11/2012-11-6.csv
Processing corpus_data/lipad/2012/11/2012-11-7.csv
Processing corpus_data/lipad/2012/11/2012-11-5.csv
Processing corpus_data/lipad/2012/11/2012-11-29.csv
Processing corpus_data/lipad/2012/11/2012-11-28.csv
Processing corpus_data/lipad/2012/11/2012-11-1.csv
Processing corpus_data/lipad/2012/11/2012-11-2.csv
Processing corpus_data/lipad/2012/6/2012-6-21.csv
Processing corpus_data/l

Processing corpus_data/lipad/1944/8/1944-8-12.csv
Processing corpus_data/lipad/1944/4/1944-4-20.csv
Processing corpus_data/lipad/1944/4/1944-4-21.csv
Processing corpus_data/lipad/1944/4/1944-4-25.csv
Processing corpus_data/lipad/1944/4/1944-4-18.csv
Processing corpus_data/lipad/1944/4/1944-4-24.csv
Processing corpus_data/lipad/1944/4/1944-4-27.csv
Processing corpus_data/lipad/1944/4/1944-4-17.csv
Processing corpus_data/lipad/1944/4/1944-4-28.csv
Processing corpus_data/lipad/1944/3/1944-3-9.csv
Processing corpus_data/lipad/1944/3/1944-3-16.csv
Processing corpus_data/lipad/1944/3/1944-3-17.csv
Processing corpus_data/lipad/1944/3/1944-3-29.csv
Processing corpus_data/lipad/1944/3/1944-3-28.csv
Processing corpus_data/lipad/1944/3/1944-3-14.csv
Processing corpus_data/lipad/1944/3/1944-3-10.csv
Processing corpus_data/lipad/1944/3/1944-3-13.csv
Processing corpus_data/lipad/1944/3/1944-3-23.csv
Processing corpus_data/lipad/1944/3/1944-3-20.csv
Processing corpus_data/lipad/1944/3/1944-3-21.csv
P

Processing corpus_data/lipad/1943/2/1943-2-9.csv
Processing corpus_data/lipad/1943/2/1943-2-8.csv
Processing corpus_data/lipad/1943/5/1943-5-12.csv
Processing corpus_data/lipad/1943/5/1943-5-6.csv
Processing corpus_data/lipad/1943/5/1943-5-7.csv
Processing corpus_data/lipad/1943/5/1943-5-13.csv
Processing corpus_data/lipad/1943/5/1943-5-11.csv
Processing corpus_data/lipad/1943/5/1943-5-10.csv
Processing corpus_data/lipad/1943/5/1943-5-14.csv
Processing corpus_data/lipad/1943/5/1943-5-28.csv
Processing corpus_data/lipad/1943/5/1943-5-17.csv
Processing corpus_data/lipad/1943/5/1943-5-27.csv
Processing corpus_data/lipad/1943/5/1943-5-26.csv
Processing corpus_data/lipad/1943/5/1943-5-18.csv
Processing corpus_data/lipad/1943/5/1943-5-24.csv
Processing corpus_data/lipad/1943/5/1943-5-31.csv
Processing corpus_data/lipad/1943/5/1943-5-25.csv
Processing corpus_data/lipad/1943/5/1943-5-19.csv
Processing corpus_data/lipad/1943/5/1943-5-21.csv
Processing corpus_data/lipad/1943/5/1943-5-20.csv
Proc

...

Processing corpus_data/lipad/1909/3/1909-3-18.csv
Processing corpus_data/lipad/1909/3/1909-3-24.csv
Processing corpus_data/lipad/1909/3/1909-3-30.csv
Processing corpus_data/lipad/1909/3/1909-3-29.csv
Processing corpus_data/lipad/1909/3/1909-3-15.csv
Processing corpus_data/lipad/1909/3/1909-3-16.csv
Processing corpus_data/lipad/1909/3/1909-3-17.csv
Processing corpus_data/lipad/1909/3/1909-3-12.csv
Processing corpus_data/lipad/1909/3/1909-3-11.csv
Processing corpus_data/lipad/1909/3/1909-3-3.csv
Processing corpus_data/lipad/1909/3/1909-3-2.csv
Processing corpus_data/lipad/1909/3/1909-3-1.csv
Processing corpus_data/lipad/1909/3/1909-3-5.csv
Processing corpus_data/lipad/1909/3/1909-3-4.csv
Processing corpus_data/lipad/1909/12/1909-12-10.csv
Processing corpus_data/lipad/1909/12/1909-12-3.csv
Processing corpus_data/lipad/1909/12/1909-12-2.csv
Processing corpus_data/lipad/1909/12/1909-12-13.csv
Processing corpus_data/lipad/1909/12/1909-12-17.csv
Processing corpus_data/lipad/1909/12/1909-12-6.

Processing corpus_data/lipad/1938/6/1938-6-10.csv
Processing corpus_data/lipad/1938/6/1938-6-13.csv
Processing corpus_data/lipad/1938/6/1938-6-17.csv
Processing corpus_data/lipad/1938/6/1938-6-16.csv
Processing corpus_data/lipad/1938/6/1938-6-28.csv
Processing corpus_data/lipad/1938/6/1938-6-14.csv
Processing corpus_data/lipad/1938/6/1938-6-15.csv
Processing corpus_data/lipad/1938/6/1938-6-29.csv
Processing corpus_data/lipad/1938/6/1938-6-8.csv
Processing corpus_data/lipad/1938/6/1938-6-9.csv
Processing corpus_data/lipad/1938/1/1938-1-28.csv
Processing corpus_data/lipad/1938/1/1938-1-31.csv
Processing corpus_data/lipad/1938/1/1938-1-27.csv
Processing corpus_data/lipad/1938/4/1938-4-6.csv
Processing corpus_data/lipad/1938/4/1938-4-7.csv
Processing corpus_data/lipad/1938/4/1938-4-5.csv
Processing corpus_data/lipad/1938/4/1938-4-4.csv
Processing corpus_data/lipad/1938/4/1938-4-1.csv
Processing corpus_data/lipad/1938/4/1938-4-26.csv
Processing corpus_data/lipad/1938/4/1938-4-25.csv
Process

Processing corpus_data/lipad/2005/3/2005-3-23.csv
Processing corpus_data/lipad/2005/3/2005-3-24.csv
Processing corpus_data/lipad/2005/2/2005-2-4.csv
Processing corpus_data/lipad/2005/2/2005-2-7.csv
Processing corpus_data/lipad/2005/2/2005-2-3.csv
Processing corpus_data/lipad/2005/2/2005-2-2.csv
Processing corpus_data/lipad/2005/2/2005-2-1.csv
Processing corpus_data/lipad/2005/2/2005-2-23.csv
Processing corpus_data/lipad/2005/2/2005-2-22.csv
Processing corpus_data/lipad/2005/2/2005-2-21.csv
Processing corpus_data/lipad/2005/2/2005-2-25.csv
Processing corpus_data/lipad/2005/2/2005-2-18.csv
Processing corpus_data/lipad/2005/2/2005-2-24.csv
Processing corpus_data/lipad/2005/2/2005-2-16.csv
Processing corpus_data/lipad/2005/2/2005-2-17.csv
Processing corpus_data/lipad/2005/2/2005-2-15.csv
Processing corpus_data/lipad/2005/2/2005-2-14.csv
Processing corpus_data/lipad/2005/2/2005-2-10.csv
Processing corpus_data/lipad/2005/2/2005-2-11.csv
Processing corpus_data/lipad/2005/2/2005-2-9.csv
Proces

Processing corpus_data/lipad/2003/9/2003-9-15.csv
Processing corpus_data/lipad/2003/9/2003-9-25.csv
Processing corpus_data/lipad/2003/9/2003-9-19.csv
Processing corpus_data/lipad/2003/9/2003-9-18.csv
Processing corpus_data/lipad/2003/9/2003-9-24.csv
Processing corpus_data/lipad/2003/9/2003-9-30.csv
Processing corpus_data/lipad/2003/9/2003-9-26.csv
Processing corpus_data/lipad/2003/9/2003-9-23.csv
Processing corpus_data/lipad/2003/9/2003-9-22.csv
Processing corpus_data/lipad/2003/11/2003-11-6.csv
Processing corpus_data/lipad/2003/11/2003-11-7.csv
Processing corpus_data/lipad/2003/11/2003-11-5.csv
Processing corpus_data/lipad/2003/11/2003-11-4.csv
Processing corpus_data/lipad/2003/11/2003-11-3.csv
Processing corpus_data/lipad/2003/6/2003-6-9.csv
Processing corpus_data/lipad/2003/6/2003-6-6.csv
Processing corpus_data/lipad/2003/6/2003-6-5.csv
Processing corpus_data/lipad/2003/6/2003-6-4.csv
Processing corpus_data/lipad/2003/6/2003-6-11.csv
Processing corpus_data/lipad/2003/6/2003-6-10.csv

Processing corpus_data/lipad/2004/3/2004-3-31.csv
Processing corpus_data/lipad/2004/12/2004-12-8.csv
Processing corpus_data/lipad/2004/12/2004-12-9.csv
Processing corpus_data/lipad/2004/12/2004-12-1.csv
Processing corpus_data/lipad/2004/12/2004-12-2.csv
Processing corpus_data/lipad/2004/12/2004-12-3.csv
Processing corpus_data/lipad/2004/12/2004-12-7.csv
Processing corpus_data/lipad/2004/12/2004-12-6.csv
Processing corpus_data/lipad/2004/12/2004-12-10.csv
Processing corpus_data/lipad/2004/12/2004-12-13.csv
Processing corpus_data/lipad/2004/12/2004-12-15.csv
Processing corpus_data/lipad/2004/12/2004-12-14.csv
Processing corpus_data/lipad/2004/2/2004-2-3.csv
Processing corpus_data/lipad/2004/2/2004-2-2.csv
Processing corpus_data/lipad/2004/2/2004-2-6.csv
Processing corpus_data/lipad/2004/2/2004-2-5.csv
Processing corpus_data/lipad/2004/2/2004-2-4.csv
Processing corpus_data/lipad/2004/2/2004-2-23.csv
Processing corpus_data/lipad/2004/2/2004-2-20.csv
Processing corpus_data/lipad/2004/2/2004

In [3]:
# just checking
pd.read_csv('corpus_data/lipad/2004/5/2004-5-12.csv')

,basepk,hid,speechdate,pid,opid,speakeroldname,speakerposition,maintopic,subtopic,subsubtopic,speechtext,speakerparty,speakerriding,speakername,speakerurl,speechtext_preprocessed
0,4277831,ca.proc.d.2004-05-12.1669.0,2004-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,As is our practice on Wednesday we will now si...,NaN,NaN,The Speaker,NaN,practice wednesday sing led edmonton north edi...
1,4277832,ca.proc.d.2004-05-12.1669.1,2004-05-12,328b9cb9-24ea-4602-abb8-ff33227c2f3f,4458.0,"Hon. Charles Caccia (Davenport, Lib.)",NaN,Statements by Members,Hybrid Fuelled Vehicles,NaN,"Mr. Speaker, a few days ago in Edmonton a new ...",Liberal,Davenport,Charles Caccia,http://www.parl.gc.ca/parlinfo/Files/Parliamen...,days ago edmonton new diesel electric hybrid b...
2,4277833,ca.proc.d.2004-05-12.1669.2,2004-05-12,c4415db6-a613-497f-8b0b-cc64d3647d04,3028.0,"Mr. Dale Johnston (Wetaskiwin, CPC)",NaN,Statements by Members,Cattle Industry,NaN,"Mr. Speaker, May 20 will mark the first annive...",Canadian Alliance,Wetaskiwin,Dale Johnston,http://www.parl.gc.ca/parlinfo/Files/Parliamen...,mark anniversary discovery bse alberta cow out...
3,4277834,ca.proc.d.2004-05-12.1669.3,2004-05-12,476c3953-0211-47f5-9f19-152383e70835,4718.0,"Ms. Sarmite Bulte (Parkdale—High Park, Lib.)",NaN,Statements by Members,Polish Second Corps,NaN,"Mr. Speaker, on Sunday, May 9 I had the honour...",Liberal,Parkdale--High Park,Sarmite Bulte,http://www.parl.gc.ca/parlinfo/Files/Parliamen...,sunday honour attending commemoration formatio...
4,4277835,ca.proc.d.2004-05-12.1669.4,2004-05-12,cbf5ad4e-06cd-4f2c-97c0-f0864c44f24e,3709.0,Mr. Christian Jobin (Lévis-et-Chutes-de-la-Cha...,NaN,Statements by Members,Marine Security,NaN,"Mr. Speaker, I would like to congratulate the ...",Liberal,Lévis-Et-Chutes-De-La-Chaudière,Christian Jobin,NaN,like congratulate marine security initiative a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,4278050,ca.proc.d.2004-05-12.1669.219,2004-05-12,37b14340-0543-4dd1-86cd-0f5dfbe99332,3465.0,"Ms. Marlene Catterall (Ottawa West—Nepean, Lib.)",NaN,Adjournment Proceedings,Excise Tax Act,NaN,"Mr. Speaker, on April 30 I asked a question of...",Liberal,Ottawa West--Nepean,Marlene Catterall,http://www.parl.gc.ca/parlinfo/Files/Parliamen...,april parliamentary public safety emergency pr...
220,4278051,ca.proc.d.2004-05-12.1669.220,2004-05-12,8df859fa-8815-43d8-b8f8-545fe2db7c49,4925.0,Hon. Yvon Charbonneau (Parliamentary Secretary...,NaN,Adjournment Proceedings,Excise Tax Act,NaN,"Mr. Speaker, I rise today in response to the r...",Liberal,Anjou--Rivière-Des-Prairies,Yvon Charbonneau,http://www.parl.gc.ca/parlinfo/Files/Parliamen...,rise today response request ottawa westnepean ...
221,4278052,ca.proc.d.2004-05-12.1669.221,2004-05-12,37b14340-0543-4dd1-86cd-0f5dfbe99332,3465.0,Ms. Marlene Catterall,NaN,Adjournment Proceedings,Excise Tax Act,NaN,"Mr. Speaker, I thank the parliamentary secreta...",Liberal,Ottawa West--Nepean,Marlene Catterall,http://www.parl.gc.ca/parlinfo/Files/Parliamen...,thank parliamentary additional response say fu...
222,4278053,ca.proc.d.2004-05-12.1669.222,2004-05-12,8df859fa-8815-43d8-b8f8-545fe2db7c49,4925.0,Hon. Yvon Charbonneau,NaN,Adjournment Proceedings,Excise Tax Act,NaN,"Mr. Speaker, I would like to remind the hon. m...",Liberal,Anjou--Rivière-Des-Prairies,Yvon Charbonneau,http://www.parl.gc.ca/parlinfo/Files/Parliamen...,like remind understandably concerned disturbin...
